In [1]:
# Accessing Data from XLM Catalog via Siphon Libraries
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

In [2]:
from datetime import datetime, timedelta

In [3]:
now = datetime.utcnow()

In [11]:
list(ncss.thredds_model_dict.keys())

['RAP', 'GFS', 'HRRR', 'GEFS', 'NAM', 'NDFD', 'RTMA', 'NCEP Blend']

In [4]:
# Request the GFS data from the thredds server
gfs_url = f"https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_{now.year}{now.month:02d}{now.day:02d}_0000.grib2/catalog.xml"
gfs = TDSCatalog(gfs_url)
    
dataset = list(gfs.datasets.values())[0]
#print(dataset.access_urls)
    
# Create NCSS object to access the NetcdfSubset
ncss = NCSS(dataset.access_urls['NetcdfSubset'])

In [5]:
now = datetime.utcnow()
# define time range you want the data for
start = now
print(start)
delta_t = 12
end = now + timedelta(hours=delta_t)

2020-10-07 21:43:11.616876


In [6]:
query = ncss.query()
query.time_range(start, end)
Lat = query.lonlat_box(north=80, south=0, east=310, west=200)
query.accept('netcdf4')

time_start=2020-10-07T21%3A43%3A11.616876&time_end=2020-10-08T09%3A43%3A11.616876&west=200&east=310&south=0&north=80&accept=netcdf4

In [7]:
mslp_name = "" # What is the name in the catalog???????
# .
# .
# .
# check out the ncss.thredds_model_dict dictionary and ncss.open_var_browser() method
ncss.thredds_model_dict["GFS"]

{'0p25_ana': 'GFS/Global_0p25deg_ana/GFS_Global_0p25deg_ana',
 '0p25': 'GFS/Global_0p25deg/GFS_Global_0p25deg',
 '0p5_ana': 'GFS/Global_0p5deg_ana/GFS_Global_0p5deg_ana',
 '0p5': 'GFS/Global_0p5deg/GFS_Global_0p5deg',
 'onedeg_ana': 'GFS/Global_onedeg_ana/GFS_Global_onedeg_ana',
 'onedeg': 'GFS/Global_onedeg/GFS_Global_onedeg',
 'Pac_20km': 'GFS/Pacific_20km/GFS_Pacific_20km',
 'PR_0p25': 'GFS/Puerto_Rico_0p25deg/GFS_Puerto_Rico_0p25deg',
 'CONUS_95km': 'GFS/CONUS_95km/GFS_CONUS_95km',
 'CONUS_80km': 'GFS/CONUS_80km/GFS_CONUS_80km',
 'CONUS_20km': 'GFS/CONUS_20km/GFS_CONUS_20km',
 'AK_20km': 'GFS/Alaska_20km/GFS_Alaska_20km'}

In [ ]:
current_catalog = ncss.open_var_browser("GFS","0p25",datetime.utcnow(),"0000")

In [8]:
mslp_name = "MSLP_Eta_model_reduction_msl"
query.variables(mslp_name).add_lonlat(True)

var=MSLP_Eta_model_reduction_msl&time_start=2020-10-07T21%3A43%3A11.616876&time_end=2020-10-08T09%3A43%3A11.616876&west=200&east=310&south=0&north=80&accept=netcdf4&addLatLon=True

In [9]:
# Request data for the variables you want to use
data = ncss.get_data(query)

In [10]:
# don't have netcdf4 installed at the moment so data wont be pulled correctly
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Originating_or_generating_Center: US National Weather Service, National Centres for Environmental Prediction (NCEP)
    Originating_or_generating_Subcenter: 0
    GRIB_table_version: 2,1
    Type_of_generating_process: Forecast
    Analysis_or_forecast_generating_process_identifier_defined_by_originating_centre: Analysis from GFS (Global Forecast System)
    Conventions: CF-1.6
    history: Read using CDM IOSP GribCollection v3
    featureType: GRID
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)
Original Dataset = /data/ldm/pub/native/grid/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_20201007_0000.grib2.ncx3#LatLon_721X1440-p125S-180p0E; Translation Date = 2020-10-07T21:43:19.841Z
    geospatial_lat_min: 0.0
    geospatial_lat_max: 80.0
    geospatial_lon_min: -160.0
    geospatial_lon_max: -50.0
    dimensions(sizes): time(4), lat(321), lon(441)
    variables(dimen

In [ ]:
ncss.thredds_model_dict

In [ ]:
list(ncss.thredds_model_dict.keys())

In [ ]:
ncss.thredds_model_dict["RAP"].keys()

In [ ]:
current_catalog = ncss.open_var_browser(model="RAP",prod="CONUS_13km",datetime_obj=datetime.utcnow(),init_hour = "0000")

In [ ]:
current_catalog

In [ ]:
ncss.open_var_browser?